In [29]:
# !pip install simpletransformers

In [30]:
# !unzip Combined\ Data.csv.zip

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [32]:
dataset=pd.read_csv('Combined Data.csv')
dataset.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [33]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53043 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53043 non-null  int64 
 1   statement   52681 non-null  object
 2   status      53043 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [34]:
dataset.isnull().sum()

,0
Unnamed: 0,0
statement,362
status,0


In [35]:
dataset['status'].unique()

array(['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar',
       'Personality disorder'], dtype=object)

In [36]:
dataset=dataset.dropna()

In [37]:
dataset.drop(columns='Unnamed: 0', inplace=True)

<ipython-input-37-b496501a1d73>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop(columns='Unnamed: 0', inplace=True)


In [38]:
dataset['labels']=dataset['status'].map({'Anxiety':0, 'Normal':1, 'Depression':2, 'Suicidal':3, 'Stress':4, 'Bipolar':5,
       'Personality disorder':6})

<ipython-input-38-d704308aa942>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['labels']=dataset['status'].map({'Anxiety':0, 'Normal':1, 'Depression':2, 'Suicidal':3, 'Stress':4, 'Bipolar':5,


In [39]:
dataset.tail()

,statement,status,labels
53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety,0
53039,"selfishness ""I don't feel very good, it's lik...",Anxiety,0
53040,Is there any way to sleep better? I can't slee...,Anxiety,0
53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety,0
53042,I have really bad door anxiety! It's not about...,Anxiety,0


In [40]:
train_df, test_df=train_test_split(dataset[['statement', 'labels']], test_size=0.2, random_state=42)

In [41]:
train_df.columns=["text", "labels"]
test_df.columns=["text", "labels"]

In [42]:
model_args = ClassificationArgs(num_train_epochs=1, output_dir='./fine_tuned')

# Create a ClassificationModel
model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=7,
    args=model_args
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

# Make predictions with the model
predictions, raw_outputs = model.predict(["I wanna commit suicide"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

  0%|          | 0/21 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/106 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
predictions[0]

3

In [46]:
#Perform Inference
model = ClassificationModel("roberta", "fine_tuned", use_cuda=True)

text_to_classify = ["I feel like dying", "I feel happy today", "I wish I was never born", "Life is a suffering"]
predictions, raw_outputs = model.predict(text_to_classify)

sentiments=['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar',
       'Personality disorder']

for i in predictions:
  print(sentiments[i])

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Suicidal
Normal
Suicidal
Normal


In [51]:
input_text="""
Neighbours Hello I’m haveing anxiety because my neighbours pour things like
 pee dog poo etc I can’t put my plants out side cuz water is always falling of their
 top balcony and on to mines there are really hostile to my then I get racism from them keep in mind
 I did nothing to them it is like they are heart less and my boys and I are scared cuz they threatened as
 I call police so much time they do nothing I’m also looking for i new home they do drugs they
 spat at us when we go out side and they say the reason why water falls from the porch is because the water
 tank they party ever day it’s really scary In hope I can move to better area I stay strong in front of my
 boys but i cry in the in side it feels like no cares about us at all it’s more worrying cuz
  I’m I single mother of 2 and upstairs is all men idk what to and I’m scared for my life and my children life
btw I’m in Scotland sorry if I mis spelled some things it cuz I’m shaking.
"""

new_predictions, raw_outputs=model.predict(input_text)

print(sentiments[new_predictions[0]])

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Anxiety
